# Before your start with this Tutorial

**Tutorial Intention:** Providing an example of iteration and related step on a modeling phase for you to:

*   Experience the data science lifecycle using Vectice
*   See how simple it is to connect your notebook to Vectice
*   Learn how to structure and log your work using Vectice

**Resources needed:**
*   <b>Tutorial Project: Forecast in-store unit sales (23.1)</b> - You can find it as part of your personal workspace       

**Other resources:**
*   Vectice Webapp Documentation: https://docs.vectice.com/
*   Vectice API documentation: https://api-docs.vectice.com/sdk/index.html

# 1. Getting Started         

**First, we need to install and authenticate ourselves to the Vectice server. Before proceeding further:**
*   Visit the Vectice app (https://app.vectice.com/account/api-keys) to create and download an API token, name the file as "My Token"
*   Upload the file to Colab by clicking on the "folder" icon on the left-hand taskbar and selecting "Upload to Session Storage"

* If you then execute

In [ ]:
%pip install --q vectice==23.1.7.2
import vectice as vct

vec = vct.connect(config="My Token.json")

#### You have successfully installed Vectice in your notebook and connected to your instance. 
#### Wasn't that easy?

## Install optional packages for your project

In [ ]:
%pip install --q squarify
%pip install --q plotly
%pip install --q matplotlib -U 

## Import packages

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Importing the relevant libraries
import IPython.display
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from matplotlib import pyplot as plt
import os
# D3 modules
from IPython.display import display
import datetime as dt
# sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
# Vectice
import vectice
from vectice import FileResource

## Reading the data

The dataset used in this project can be found here:<br>
* [items.csv](https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/items.csv)<br>
* [holidays_events.csv](https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/holidays_events.csv)<br>
* [stores.csv](https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/stores.csv)<br>
* [oil.csv](https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/oil.csv)<br>
* [transactions.csv](https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/transactions.csv)<br>
* [train_reduced.csv](https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/train_reduced.csv)

Excute the cell below to download the files locally

In [ ]:
# Download the files locally
!wget https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/items.csv -q --no-check-certificate
!wget https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/holidays_events.csv -q --no-check-certificate
!wget https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/stores.csv -q --no-check-certificate
!wget https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/oil.csv -q --no-check-certificate
!wget https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/transactions.csv -q --no-check-certificate
!wget https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/train_reduced.csv -q --no-check-certificate

#### Great! Let's build dataframes from the file for later use

In [ ]:
dtypes = {'store_nbr': np.dtype('int64'),
          'item_nbr': np.dtype('int64'),
          'unit_sales': np.dtype('float64'),
          'onpromotion': np.dtype('O')}

items = pd.read_csv("items.csv")
holiday_events = pd.read_csv("holidays_events.csv", parse_dates=['date'])
stores = pd.read_csv("stores.csv")
oil = pd.read_csv("oil.csv", parse_dates=['date'])
transactions = pd.read_csv("transactions.csv", parse_dates=['date'])
train = pd.read_csv("train_reduced.csv", parse_dates=['date'], on_bad_lines='warn')

# Feature engineering

**Here we analyze the data and select the features for our model to be trained on.**

**Train**
id, date, store_nbr, item_nbr, unit_scale, on_promotion

**Items**
item_nbr, family, class, perishable

**Holidays_events**
date, type, locale, locale_name, description, transferred

**Stores**
store_nbr, city, state, type, cluster

**Oil**
date, dcoilwtico

**Transactions**
date, store_nbr, transactions

**Selected features as inputs to the model**

date, holiday.type, holidaye.locale, holiday.locale_name, holiday_transfered, store_nbr, store.city, store.state, store.type, store.cluster, transactions, item_nbr, item.family, item.class, on_promotion, perishable, dcoilwtico.

**Selected features as outputs of the model**

transactions per store, unit_sales per item

## DATA pipeline

#### The next four cells are functions used as part of our Data Pipeline process.
#### Nothing Vectice specific, just boiler plate code.
#### Feel free to look through it but no need to spend time on it.
#### Go ahead and jump ahead to "Documeetn in Vection" section below

In [ ]:
class prepare_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("prepare_data -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        train_stores = X[0].merge(X[1], right_on = 'store_nbr', left_on='store_nbr')
        train_stores_oil = train_stores.merge(X[2], right_on='date', left_on='date')
        train_stores_oil_items = train_stores_oil.merge(X[3], right_on = 'item_nbr', left_on = 'item_nbr')
        train_stores_oil_items_transactions = train_stores_oil_items.merge(X[4], right_on = ['date', 'store_nbr'], left_on = ['date', 'store_nbr'])
        train_stores_oil_items_transactions_hol = train_stores_oil_items_transactions.merge(X[5], right_on = 'date', left_on = 'date')
        
        data_df = train_stores_oil_items_transactions_hol.copy(deep = True)
        
        # Fill the empty values
        data_df['onpromotion'] = data_df['onpromotion'].fillna(0)
        # change the bool to int
        data_df['onpromotion'] = data_df['onpromotion'].astype(int)
        data_df['transferred'] = data_df['transferred'].astype(int)

        # change the names
        data_df.rename(columns={'type_x': 'st_type', 'type_y': 'hol_type'}, inplace=True)
        
        # handle date
        data_df['date'] = pd.to_datetime(data_df['date'])
        data_df['date'] = data_df['date'].map(dt.datetime.toordinal)
                
        return data_df

### Custom transform for splitting the data

Here, we split dataframe into numerical values, categorical values and date

In [ ]:
# split dataframe into numerical values, categorical values and date
class split_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("split_data -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # Get columns for each type         
        df_ = X.drop(['date'], axis = 1)
        cols = df_.columns
        num_cols = df_._get_numeric_data().columns
        cat_cols = list(set(cols) - set(num_cols))
        
        data_num_df = X[num_cols]
        data_cat_df = X[cat_cols]
        data_date_df = X['date']
        
        return data_num_df, data_cat_df, data_date_df

Here, we handle the missing data, apply standard scaler to numerical attributes, and convert categorical data into numerical

In [ ]:
class process_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("process_data -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        ### numerical data
        # impute nulls in numerical attributes
        imputer = SimpleImputer(strategy="mean", copy=True)
        num_imp = imputer.fit_transform(X[0])
        #########
        data_num_df = pd.DataFrame(num_imp, columns=X[0].columns, index=X[0].index)
        
        # apply standard scaling
        scaler = StandardScaler()
        scaler.fit(data_num_df)
        num_scaled = scaler.transform(data_num_df)
        data_num_df = pd.DataFrame(num_scaled, columns=X[0].columns, index=X[0].index)
        
        ### categorical data
        # one hot encoder
        cat_encoder = OneHotEncoder(sparse=False)
        data_cat_1hot = cat_encoder.fit_transform(X[1])
        
        # convert it to dataframe with n*99 where n number of rows and 99 is no. of categories
        data_cat_df = pd.DataFrame(data_cat_1hot, columns=cat_encoder.get_feature_names_out()) #, index=X[1].index)
                
        return data_num_df, data_cat_df, X[2]

In [ ]:
class join_df(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("join_df -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        ### numerical data
        data_df = X[0].join(X[1])
        data_df = data_df.join(X[2])
        
        return data_df

# Push the datasets through the pipeline

In [ ]:
pipe_processing = Pipeline([
        ('prepare_data', prepare_data()),
        ('split_data', split_data()),
        ('process_data', process_data()),
        ('join_data', join_df())
    ])

# our prepared data
data_df = pipe_processing.fit_transform([train, stores, oil, items, transactions, holiday_events])
data_df.to_csv("train_clean.csv") #this is the dataset that will be split into a training, testing, and validation dataset

#### Navigate your way to your personal workspace, get the tutorial project and start an iteration of the 'Data Prep' phase. Go ahead and execute the cell below to navigate to your workspace. 

In [ ]:
active_iter = vec.workspaces[0].project(8055).phase("Data Preparation").iteration() # Make sure to use the proper project ID or name

## Capture milestones for the Data Preparation phase

#### Execute the following cell

In [ ]:
# Provide context into the origin datasets by attaching them to the step
active_iter.step_select_data = vct.Dataset.origin(name="Items origin",resource=FileResource(path="items.csv"))
active_iter.step_select_data += vct.Dataset.origin(name="Holiday origin",resource=FileResource(path="holidays_events.csv"))
active_iter.step_select_data += vct.Dataset.origin(name="Stores origin",resource=FileResource(path="stores.csv"))
active_iter.step_select_data += vct.Dataset.origin(name="Oil origin",resource=FileResource(path="oil.csv"))
active_iter.step_select_data += vct.Dataset.origin(name="Transactions origin",resource=FileResource(path="transactions.csv"))

active_iter.step_select_data = "The datasets for the project have been identified"

# Great we have documented the datasets used.

# Let's move on the next step...documenting our data pipeline
# Log in findings/comments for this milestone
msg = "As part of our standard Data Pipeline process we applied the following preparation to our datasets:\n - Handling of missing data\n - Applied standard scaler to numerical attributes\n - Converted categorical data into numerical\n - Split values in numerical values, categorical values, and dates"
active_iter.step_clean_data = msg

# Log in findings/comments for this milestone, close the step and capture the next one
active_iter.step_construct_data = "We selected \"unit sales\" as our model target.\nThe features used in this model are:\n - date\n - holiday.type\n - holidaye.locale\n - holiday.locale_name\n - holiday_transfered\n - store_nbr\n - store.city\n - store.state\n - store.type\n - store.cluster\n - transactions\n - item_nbr\n - item.family\n - item.class\n - on_promotion\n - perishable\n - dcoilwtico"

# Log in findings/comments for this milestone, close the step and capture the next one'
# and attach the clean dataset generated
msg = "We processed our origin datasets through our data pipeline to generate a dataset ready for modeling.\n"
msg += f"The resulting modeling datasets contains {data_df.shape[0]} observations and {data_df.shape[1]} features.\n"
msg += "The dataset is ready to be split for modeling."
active_iter.step_integrate_data = vct.Dataset.clean(name="Clean&Augmented_Dataset",resource=FileResource(path="train_clean.csv"))
active_iter.step_integrate_data = msg

# Log in our activity
active_iter.step_format_data = "We generated a dataset ready for modeling. We also created a data pipeline to make this process repeatable."
active_iter.complete()

#### The cell above creates a new phase iteration and document all the step in our data pipeline process